In [17]:
from pathlib import Path
from collections import defaultdict, Counter
import numpy as np
ninf = -1e8

In [18]:
def read_data(path, column=0):
    """column=0 means input sequence, column=1 means label
    """
    with open(path) as f:
        lines = f.readlines()
    
    data = []
    sample = []
    
    for line in lines:
        formatted_line = line.strip()
        
        if len(formatted_line) > 0:
            split_data = formatted_line.split(" ")
            sample.append(split_data[column])

        else:
            data.append(sample)
            sample = []
            
    return data

In [19]:
full_dir = Path('full')
partial_dir = Path('partial')

x_data, y_data = read_data(partial_dir/'train', 0), read_data(partial_dir/'train', 1)

In [20]:
# number of instances in the dataset
len(x_data), len(y_data)

(700, 700)

In [21]:
# y labels
y_vocab = list(set([oo for o in y_data for oo in o])); y_vocab

['B-art',
 'B-gpe',
 'I-org',
 'B-nat',
 'I-eve',
 'I-geo',
 'O',
 'I-gpe',
 'B-eve',
 'B-geo',
 'I-tim',
 'I-art',
 'B-per',
 'I-per',
 'B-tim',
 'B-org']

In [22]:
# x vocab
x_vocab = list(set([oo for o in x_data for oo in o])); len(x_vocab)

4068

## Part I (i): Emission scores

In [23]:
def calc_e(x_data, y_data, x_vocab, y_vocab):
    count_emission = Counter([(x,y) for x_instance, y_instance in zip(x_data, y_data) for x, y in zip(x_instance, y_instance)])
    count_label = Counter([oo for o in y_data for oo in o])
    
    e_score = {}
    for y in y_vocab:
        for x in x_vocab:
            feature = f"emission:{y}+{x}"
            
            if (x,y) not in count_emission:
                e_score[feature] = ninf
            else:
                score = np.log(count_emission[(x,y)]  /  count_label[y])
                e_score[feature] = score
    
    return e_score

In [24]:
emission_dict = calc_e(x_data, y_data, x_vocab, y_vocab)

## Part I (ii): Transition scores

In [25]:
def calc_t(y_data, y_vocab):
    count_transition = Counter([ (y_prev, y) for y_instance in y_data for y_prev, y in zip(['START'] + y_instance, y_instance + ['STOP'])])
    count_label = Counter([y for y_instance in y_data for y in ['START'] + y_instance])
    
    f_score = {}
    for y_prev in ['START'] + y_vocab:
        for y in y_vocab + ['STOP']:
            feature = f"transition:{y_prev}+{y}"
            
            if (y_prev,y) not in count_transition:
                f_score[feature] = ninf
            else:
                score = np.log(count_transition[(y_prev,y)]  /  count_label[y_prev])
                f_score[feature] = score
    
    return f_score

In [26]:
transition_dict = calc_t(y_data, y_vocab)

In [27]:
feature_dict = {**transition_dict, **emission_dict}

## Part II (i): Compute Score

In [28]:
def compute_score(x_instance, y_instance, feature_dict):
    feature_count = defaultdict(int)
    
    for x, y in zip(x_instance, y_instance): feature_count[f"emission:{y}+{x}"] += 1
    
    for y_prev, y in zip(['START'] + y_instance, y_instance + ['STOP']):
        feature_count[f"transition:{y_prev}+{y}"] += 1
        
    score = sum([feature_dict[feat]*count for feat, count in feature_count.items()])
    return score

In [29]:
x_instance = "This is the second U.N.-Congolese offensive against militias in the region since the DRC 's constitutional referendum a week ago .".split()
y_instance = 'O O O O O O O O O O O O O B-geo O O O O O O O'.split()
compute_score(x_instance, y_instance, feature_dict)

-139.57175855522826

## Part II (ii): Viterbi decoding

### TODO: 
how to evaluate using conlleval.evaluate function

do evaluation on the dev set

In [30]:
def viterbi(x_instance, y_vocab, feature_dict):
    n, d = len(x_instance), len(y_vocab)
    scores = np.full( (n,d), ninf)
    bp = np.full( (n,d), 0, dtype=np.int)
    
    for i, y in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:START+{y}",  ninf)
        e_score = feature_dict.get( f"emission:{y}+{x_instance[0]}",  ninf)
        scores[0, i] = t_score + e_score
        
    for i in range(1, n):
        for y_i, y in enumerate(y_vocab):
            for y_prev_i, y_prev in enumerate(y_vocab):
                t_score = feature_dict.get( f"transition:{y_prev}+{y}", ninf)
                e_score = feature_dict.get( f"emission:{y}+{x_instance[i]}", ninf)
                score = t_score + e_score + scores[i-1, y_prev_i]
                if score > scores[i, y_i]:
                    scores[i, y_i] = score
                    bp[i, y_i] = y_prev_i
    
    final_score, final_bp = ninf, 0
    for i, y_prev in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:{y_prev}+STOP", ninf)
        score = t_score + scores[n-1, i]
        if score > final_score: 
            final_score = score
            final_bp = i
            
    decoded_sequence = [ y_vocab[final_bp], ]
    for i in range(n-1, 0, -1):
        final_bp = bp[i, final_bp]
        decoded_sequence = [ y_vocab[final_bp] ] + decoded_sequence
        
    return decoded_sequence

print(" ".join(viterbi(x_instance, y_vocab, feature_dict)),)
print(" ".join(y_instance))

O O O O O O O O O O O O O B-geo O O O O O O O
O O O O O O O O O O O O O B-geo O O O O O O O


## Part III (i): CRF loss
refer to [machine learning slides](https://drive.google.com/file/d/1RfPcnQigx4jdLtnTjjjI1Jgd2UufexhV/view?usp=sharing) for details

In [33]:
def logsumexp(a):
    if np.sum(a) > 100:
        b = a.max()
        return b + np.log( (np.exp(a-b)).sum() )
    if np.exp(a).sum() < 1e-100:
        return ninf
    else:
        return np.log( np.exp(a).sum() )

def forward(x_instance, y_vocab, feature_dict):
    n, d = len(x_instance), len(y_vocab)
    scores = np.full( (n,d), ninf)
    
    for i, y in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:START+{y}", ninf)
        scores[0, i] = t_score
    
    for i in range(1, n):
        for y_i, y in enumerate(y_vocab):
            temp = np.full( (d,), ninf)
            for y_prev_i, y_prev in enumerate(y_vocab):
                t_score = feature_dict.get( f"transition:{y_prev}+{y}", ninf)
                e_score = feature_dict.get( f"emission:{y_prev}+{x_instance[i-1]}", ninf)
                temp[y_prev_i] = e_score + t_score + scores[i-1, y_prev_i]
            scores[i, y_i] = logsumexp(temp)
    
    temp = np.full( (d,), ninf)
    for i, y_prev in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:{y_prev}+STOP", ninf)
        e_score = feature_dict.get( f"emission:{y_prev}+{x_instance[-1]}", ninf)
        temp[i] = e_score + t_score + scores[-1, i]
    alpha = logsumexp(temp)
    
    return scores, alpha

def loss_fn(x_instance, y_instance, feature_dict, y_vocab):
    first_term = compute_score(x_instance, y_instance, feature_dict)
    _, forward_score = forward(x_instance, y_vocab, feature_dict)
    return forward_score - first_term

In [34]:
loss_fn(x_instance, y_instance, feature_dict, y_vocab)

1.3302877199753311

## Part III (ii): forward backward for gradient

In [35]:
def backward(x_instance, y_vocab, feature_dict):
    n, d = len(x_instance), len(y_vocab)
    scores = np.full( (n,d), ninf)
    
    for i, y in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:{y}+STOP", ninf)
        e_score = feature_dict.get( f"emission:{y}+{x_instance[-1]}", ninf)
        scores[-1, j] = t_score + e_score
        
    for i in range(n-1, 0, -1):
        for y_i, y in enumerate(y_vocab):
            temp = np.full( (d,), ninf)
            for y_next_i, y_next in enumerate(y_vocab):
                t_score = feature_dict.get( f"transition:{y}+{y_next}", ninf)
                e_score = feature_dict.get( f"emission:{y}+{x_instance[i-1]}")
                temp[y_next_i] = e_score + t_score + scores[i, y_next_i]
            scores[i-1, k] = logsumexp(temp)
            
    temp = np.full( (d,), ninf)
    for i, y_next in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:START+{y_next}")
        temp[i] = t_score + scores[0, i]
    beta = logsumexp(temp)
    
    return scores, beta



def forward_backward(x_instance, y_vocab, feature_dict):
    n, d = len(x_instance), len(y_vocab)
    f_scores, alpha = forward(x_instance, y_vocab, feature_dict)
    b_scores, beta = backward(x_instance, y_vocab, feature_dict)
    
    feature_expected_count = defaultdict(float)
    
    for i in range(n):
        for y_i, y in enumerate(y_vocab):
            e_feature = f"emission:{y}+{x_instance[i]}"
            feature_expected_count[e_feature] += f_scores[i, y_i] + b_scores[i, y_i] - alpha # TODO: here do we need to add exp?
            
    for i, y_next in enumerate(y_vocab):
        t_feature = f"transition:START+{y_next}"
        feature_expected_count[t_feature] += f_scores[0, i] + b_scores[0, i] - alpha
        
        t_feature = f"transition:{y_next}+STOP"
        feature_expected_count[t_feature] += f_scores[-1, i] + b_scores[-1, i] - alpha
        
    for y_i, y in enumerate(y_vocab):
        for y_next_i, y_next in enumerate(y_vocab):
            t_feature = f"transition:{y}+{y_next}"
            total = 0
            for i in range(n-1):
                total += f_scores[i, y_i] + b_scores[i, y_i] - alpha # TODO: here do we need to include transition and emission scores?
            feature_expected_count[t_feature] = total